In [2]:
TASK_CODE = "250321-EM-001"

In [4]:
import numpy as np
import json
import pandas as pd

def convert_json_to_csv(json_file_path, output_csv_path):

    # JSON 파일 로드
    with open(json_file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    # DataFrame 변환
    df = pd.DataFrame(data)

    # 상담 텍스트(HS01, HS02, HS03만 합침) 추출
    df["text"] = df["talk"].apply(lambda x: " ".join([x["content"].get(k, "") for k in ["HS01", "HS02", "HS03"]]).strip())
    df["emotion"] = df["profile"].apply(lambda x: x["emotion"]["type"])
    df["situation"] = df["profile"].apply(lambda x: x["emotion"]["situation"][0])

    # 필요 없는 컬럼 제거
    df = df[["text", "emotion", "situation"]]

    # CSV 저장
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ 변환 완료: {output_csv_path}")


# 🔹 두 개의 JSON 파일 변환 실행
convert_json_to_csv("data/감성대화말뭉치(최종데이터)_Training.json", f"data/{TASK_CODE}_train_dataset.csv")
convert_json_to_csv("data/감성대화말뭉치(최종데이터)_Validation.json", f"data/{TASK_CODE}_test_dataset.csv")


✅ 변환 완료: data/250321-EM-001_train_dataset.csv
✅ 변환 완료: data/250321-EM-001_test_dataset.csv


In [6]:
import pandas as pd

# 데이터프레임 불러오기
train_df = pd.read_csv('data/250321-EM-001_train_dataset.csv')
test_df = pd.read_csv('data/250321-EM-001_test_dataset.csv')

# 데이터프레임 합치기
combined_df = pd.concat([train_df, test_df], ignore_index=True)

# 결과 확인
print(combined_df)


                                                    text emotion situation
0      일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...     E18       S06
1      이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....     E18       S06
2      회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...     E18       S06
3      직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...     E18       S06
4      얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...     E18       S06
...                                                  ...     ...       ...
58263  나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...     E54       S05
58264  친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아...     E54       S05
58265  남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네. ...     E54       S05
58266  나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...     E54       S05
58267  친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네. 맞아. 하지만 그렇다고 아무나하고...     E53       S05

[58268 rows x 3 columns]


In [43]:
# 감정(emotion) ID -> 한국어 라벨 매핑
emotion_map = {
    "E10": "분노", "E11": "툴툴대는", "E12": "좌절한", "E13": "짜증내는",
    "E14": "방어적인", "E15": "악의적인", "E16": "안달하는", "E17": "구역질 나는",
    "E18": "노여워하는", "E19": "성가신", "E20": "슬픔", "E21": "실망한",
    "E22": "비통한", "E23": "후회되는", "E24": "우울한", "E25": "마비된",
    "E26": "염세적인", "E27": "눈물이 나는", "E28": "낙담한", "E29": "환멸을 느끼는",
    "E30": "불안", "E31": "두려운", "E32": "스트레스 받는", "E33": "취약한",
    "E34": "혼란스러운", "E35": "당혹스러운", "E36": "회의적인", "E37": "걱정스러운",
    "E38": "조심스러운", "E39": "초조한", "E40": "상처", "E41": "질투하는",
    "E42": "배신당한", "E43": "고립된", "E44": "충격 받은", "E45": "가난한, 불우한",
    "E46": "희생된", "E47": "억울한", "E48": "괴로워하는", "E49": "버려진",
    "E50": "당황", "E51": "고립된(당황한)", "E52": "남의 시선을 의식하는", "E53": "외로운",
    "E54": "열등감", "E55": "죄책감의", "E56": "부끄러운", "E57": "혐오스러운",
    "E58": "한심한", "E59": "혼란스러운(당황한)", "E60": "기쁨", "E61": "감사하는",
    "E62": "신뢰하는", "E63": "편안한", "E64": "만족스러운", "E65": "흥분",
    "E66": "느긋", "E67": "안도", "E68": "신이난", "E69": "자신하는"
}

# 상황(situation) ID -> 한국어 라벨 매핑
situation_map = {
    "S01": "가족관계", "S02": "학업 및 진로", "S03": "학교폭력/따돌림",
    "S04": "대인관계", "S05": "연애, 결혼, 출산", "S06": "진로, 취업, 직장",
    "S07": "대인관계(부부, 자녀)", "S08": "재정, 은퇴, 노후준비", "S09": "건강",
    "S10": "직장, 업무 스트레스", "S11": "건강, 죽음", "S12": "대인관계(노년)",
    "S13": "재정"
}

# 새로운 컬럼 추가
combined_df["emotion_kor"] = combined_df["emotion"].map(emotion_map)
combined_df["situation_kor"] = combined_df["situation"].map(situation_map)



In [45]:
combined_df.head()

,text,emotion,situation,emotion_kor,situation_kor
0,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,E18,S06,노여워하는,"진로, 취업, 직장"
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,E18,S06,노여워하는,"진로, 취업, 직장"
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,E18,S06,노여워하는,"진로, 취업, 직장"
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,E18,S06,노여워하는,"진로, 취업, 직장"
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,E18,S06,노여워하는,"진로, 취업, 직장"


In [47]:
combined_df.to_csv(f'data/{TASK_CODE}-combined_df.csv', encoding='utf-8-sig', index=False)
print(f'data/{TASK_CODE}-combined_df.csv 저장완료')

data/250321-EM-001-combined_df.csv 저장완료


In [69]:
pre_df = pd.read_csv(f'data/{TASK_CODE}-combined_df.csv')
pre_df['em_kor'] = np.nan
pre_df['em_code'] = np.nan
pre_df['si_kor'] = np.nan
pre_df['si_code'] = np.nan

In [91]:
pd.set_option('display.max_colwidth', 500)

In [93]:
# 텍스트 길이 계산
pre_df['text_length'] = pre_df['text'].apply(len)

# 가장 짧은 문장의 길이
min_length = pre_df['text_length'].min()

# 가장 긴 문장의 길이
max_length = pre_df['text_length'].max()

# 전체 평균 길이
mean_length = pre_df['text_length'].mean()

min_length, max_length, mean_length

(17, 309, 100.97044690052859)

In [97]:
pre_df.sample(30)

,text,emotion,situation,emotion_kor,situation_kor,em_kor,em_code,si_kor,si_code,text_length
48933,애들만 키우면서 살았더니 내 삶이 없는 것 같아서 허무해. 애들을 위해서 열심히 일하고 평생을 살아서 내 주변엔 사람도 없고 누구와도 친하지 않아. 내 흥미를 찾아보고 주변 사람들과 취미를 공유하면서 친하게 지내려고 노력해 보려고.,E38,S04,조심스러운,대인관계,NaN,NaN,NaN,NaN,129
58123,난 참 한심해. 이제 곧 은퇴인데 은퇴 후 준비를 하나도 안 해놨어. 다시 일하는 거 외에는 다른 수가 없는 거 같아.,E58,S08,한심한,"재정, 은퇴, 노후준비",NaN,NaN,NaN,NaN,66
44844,내 몸이 성치 않아서 너무 불행해 남들처럼 평범하게 지내고 싶었는데. 건강하게 낳아주지 않았다는 생각에 부모님에게 매일 하소연 하니 부모님과 갈등도 깊어졌어. 요번 주말에 부모님과 여행을 가보자고 할까 봐 깊은 대화가 필요해.,E45,S07,"가난한, 불우한","대인관계(부부, 자녀)",NaN,NaN,NaN,NaN,126
36526,내가 원하는 대학교가 이등급이 나와야 들어갈수있대. 그래. 근데 나는 삼등급이면 통과될줄알았는데 배신당한느낌이야. 학업을 열심히 해야지. 아직 가능성 있을거같아,E42,S02,배신당한,학업 및 진로,NaN,NaN,NaN,NaN,89
8515,오늘 친구 어머님께서 밥 먹고 가라고 해서 먹는 데 기분 나빴어. 은근슬쩍 내 근황 물으면서 자기 아들하고 날 비교하면서 깎아내리더라고. 맞아. 친구 어머니라 뭐라 할 수도 없고.,E29,S04,환멸을 느끼는,대인관계,NaN,NaN,NaN,NaN,100
30246,내 성적을 보고 초조해졌어. 성적이 많이 떨어졌어.,E39,S02,초조한,학업 및 진로,NaN,NaN,NaN,NaN,28
22196,회사에 취직하게 되었는데 취직한 부서가 내가 지원했던 부서가 아니어서 혼란스러워. 나는 마케팅부서에 지원했는데 왜 경리부에 합격했는지 모르겠어. 내 전공이 아니어서 겁나. 회사 채용팀에 전화해서 사실대로 말하고 어떻게 된 상황인지 알아봐야겠어.,E35,S06,당혹스러운,"진로, 취업, 직장",NaN,NaN,NaN,NaN,136
13368,우리 반에 내가 좋아하는 남자애가 생겼는데 인기가 너무 많아. 내가 인사해도 거들떠보지도 않겠지? 인기가 너무 많으니까. 쑥스럽지만 내일 가서 먼저 인사해 봐야겠다.,E37,S02,걱정스러운,학업 및 진로,NaN,NaN,NaN,NaN,92
13573,사람들이 선남선녀라 하는 저 커플이 깨졌으면 좋겠어. 보기만 해도 너무 질투가 나서 그래. 아마도 내가 연애한 지 오래돼서 히스테리를 부리고 있나 봐.,E16,S05,안달하는,"연애, 결혼, 출산",NaN,NaN,NaN,NaN,84
12186,아무도 나랑 친구가 될 생각이 없어 보여서 불안해. 맞아. 이러다가 내 주변에 아무도 안 남으면 어쩌지? 나와 관련한 소문 때문에 아무도 나에게 다가오지 않는 것 같아.,E30,S04,불안,대인관계,NaN,NaN,NaN,NaN,94


In [99]:
pre_df.to_csv(f'data/{TASK_CODE}-combined_df_NaN.csv')